## Setup

In [ ]:
from specific import *

### Retrieve previous results from the 'model' notebook

In [ ]:
X_train, X_test, y_train, y_test = data_split_cache.load()
rf = get_model()

#### Get Dask Client

In [ ]:
def print_path():
    import sys

    return sys.path


client.submit(print_path).result()

In [ ]:
# client.scheduler_info()['workers']['tcp://10.149.10.103:40991']

In [ ]:
def p2():
    import wildfires

    return str(wildfires)


for worker in client.scheduler_info()["workers"]:
    # print(worker)
    # print(client.submit(print_path, pure=False, workers={worker}).result())
    try:
        print(client.submit(p2, pure=False, workers={worker}).result())
    except:
        print("Failed:", worker)

In [ ]:
client.restart()

In [ ]:
# client = Client(n_workers=1, threads_per_worker=8, resources={'threads': 8})
client = get_client()
client

### Dask LOCO

In [ ]:
loco_cache = SimpleCache("loco_results", cache_dir=CACHE_DIR)

leave_out = [""]
leave_out.extend(X_train.columns)

# XXX:
# loco_cache.clear()


@loco_cache
def get_loco_scores():
    return dict(
        dask_fit_loco(
            rf, X_train, y_train, client, leave_out, local_n_jobs=31, verbose=True
        )
    )


scores = get_loco_scores()

In [ ]:
scores